## Долгосрочная работа на тему граф линейного преобразования. Часть 2. Срок сдачи 15.12.2022

В данной долгосрочной работе будут исследованы некоторые свойства  подстановок из группы $AGL(n, q)$ и $GL(n, q)$, связанные с их цикловой структурой. Также в ходе работы планируется использование алгоритмов для работы с группами подстановок.

### Нумерация элементов векторных пространств

Далее мы будем работать с полями и векторными пространствами. При такой работе необходимо придерживаться некоторой естественной нумерации для элементов данных алгебраических структур. Например естественной нумерацией для элементов поля $GF(p)$ будет соответствие
$$
\begin{array}{l}
[0]_{p} \leftrightarrow 0 \\
[1]_{p} \leftrightarrow 1 \\
\ldots \\
[p-1]_{p} \leftrightarrow p-1
\end{array}
$$

Для
$GF(4)$ можно использовать нумерацию 
$$
\begin{array}{l}
[0]_{x^2+x+1} \leftrightarrow 0 \\
[1]_{x^2+x+1} \leftrightarrow 1 \\
[x]_{x^2+x+1} \leftrightarrow 2 \\
[x+1]_{x^2+x+1} \leftrightarrow 3
\end{array}
$$

Обобщим эту идею для нумерации элементов произвольного конечного поля, рассматривая данное поле как векторное пространство над своим простым подполем. То есть если $P=GF(q), q = p^n$, то 
$$
P\cong Z_p[x]/f(x), \deg f(x) = n
$$
Тогда 
$$
[a_0 + a_1x + \ldots + a_{n-1}x^{n-1}]_{f(x)} \leftrightarrow a_0 + a_1p + \ldots + a_{n-1}p^{n-1}
$$
Обозначим данное соответствие через $\mathcal{N}$.


Далее нам нужно научиться нумеровать элементы векторных пространств. Пусть $P = GF(q)$, $V = {}_PP^{(n)}$ - пространство векторов столбцов длины $n$. Будем использовать следующую нумерацию
$$
\left(
\begin{array}{l}
b_0 \\
b_1 \\
\ldots \\
b_{n-1}
\end{array}
\right)
\leftrightarrow \mathcal{N}(b_0) + \mathcal{N}(b_1)q + \ldots + \mathcal{N}(b_{n-1})q^{n-1} 
$$

~~Реализуйте данную идею для нумерации и напишите тесты~~

Ниже мы реализуем данную идею для нумерации элементов векторных пространств. Обратите внимание, на рекурсивное определение данного класса: мы используем нумерацию элементов поля как векторного пространства над своим простым подполем

In [8]:
import numpy as np # удобная библиотека для векторно-матричных вычислений

class VectorSpaceOrdering:
    def __init__(self, V):
        self.V = V
        self.P = V.base()
        
        self.card = V.cardinality()
        self.base_card = self.P.cardinality()
        self.dim =  int(log(self.card, self.base_card))
        
        self.p = self.P.base().cardinality()
        self.base_field_dim = int(log(self.base_card, self.p))
        
        self.base_ordering = VectorSpaceOrdering(self.P) if not self.base_card.is_prime() else  lambda x: self.P(x) 
        
    def _get_digits(self, idx):
        ans = []
        for _ in range(self.dim):
            ans.append(idx % self.base_card)
            idx = idx // self.base_card
        return ans
    
    def field_el_to_num(self, field_el):
        coefs = [int(el) for el in field_el.polynomial().list()]
        ans = 0
        for i, c in enumerate(coefs):
            ans += c * self.p ^ i
        return ans
    
    def vec_to_num(self, vec):
        digits = np.array([self.field_el_to_num(c) for c in vec])
        return (digits * np.array([self.base_card ^ i for i in range(self.dim)])).sum()
    
    def __call__(self, idx):
        assert idx < self.card
        digits = self._get_digits(idx)
        elems = [self.base_ordering(d) for d in digits]
        if len(elems) == 1:
            elems = elems[0]
        return self.V(elems)  

Продемонстрируем работу данного класса

In [10]:
P = GF(2)
vsordering = VectorSpaceOrdering(P)
for i in range(P.cardinality()):
    print(i, '<->', vsordering(i))

0 <-> 0
1 <-> 1


In [14]:
P = GF(9)
vsordering = VectorSpaceOrdering(P)
for i in range(P.cardinality()):
    print(i, '<->', vsordering(i))

0 <-> 0
1 <-> 1
2 <-> 2
3 <-> z2
4 <-> z2 + 1
5 <-> z2 + 2
6 <-> 2*z2
7 <-> 2*z2 + 1
8 <-> 2*z2 + 2


In [15]:
P = GF(3)
V = VectorSpace(P, 2)
vsordering = VectorSpaceOrdering(V)
for i in range(V.cardinality()):
    print(i, '<->', vsordering(i))

0 <-> (0, 0)
1 <-> (1, 0)
2 <-> (2, 0)
3 <-> (0, 1)
4 <-> (1, 1)
5 <-> (2, 1)
6 <-> (0, 2)
7 <-> (1, 2)
8 <-> (2, 2)


In [16]:
P = GF(4)
V = VectorSpace(P, 2)
vsordering = VectorSpaceOrdering(V)
for i in range(V.cardinality()):
    print(i, '<->', vsordering(i))

0 <-> (0, 0)
1 <-> (1, 0)
2 <-> (z2, 0)
3 <-> (z2 + 1, 0)
4 <-> (0, 1)
5 <-> (1, 1)
6 <-> (z2, 1)
7 <-> (z2 + 1, 1)
8 <-> (0, z2)
9 <-> (1, z2)
10 <-> (z2, z2)
11 <-> (z2 + 1, z2)
12 <-> (0, z2 + 1)
13 <-> (1, z2 + 1)
14 <-> (z2, z2 + 1)
15 <-> (z2 + 1, z2 + 1)


### Аффинные подстановки

Далее мы напишем класс реализующий аффинные подстановки. Здесь же мы реализуем разбиение подстановки на циклы.

In [17]:
class AffinePermutation:
    def __init__(self, A, b):
        assert A.is_invertible()
        self.A = A
        self.b = b
        
        self.V = b.parent()
        self.P = self.A.parent().base()
        self.n = self.A.nrows() # matrix size
        self.q = self.P.cardinality()
        
        self.space_ordering = VectorSpaceOrdering(V)
        

    def __call__(self, x):
        return A*x + b
    
    def get_cycle(self, x):
        start = x
        ans = [start]
        
        cur = x
        nxt = self(cur)
        
        while nxt != start:
            ans.append(nxt)
            cur = nxt
            nxt = self(cur)
        
        return tuple(ans)
    
    def get_cycles(self, numeric_form = False):
        cycles = []
        points = set(range(self.q ^ self.n))
        while len(points) != 0:
            start_idx = next(iter(points))
            start = self.space_ordering(start_idx)
            cycle = self.get_cycle(start)
            cycles.append(cycle)
            for p in cycle:
                p = self.space_ordering.vec_to_num(p)
                points.remove(p)
        
        if numeric_form:
            cycles_with_nums = []
            for c in cycles:
                cur = []
                for el in c:
                    cur.append(self.space_ordering.vec_to_num(el))
                cycles_with_nums.append(tuple(cur))
            
        return cycles_with_nums if numeric_form else cycles

Продемонстрируем работу данного класса. Для этого рассмотрим цикловую структуру линейного преобразования, с минимальным многочленом являющимся многочленом максимального периода.

In [21]:
P = GF(2)
n = 2
V = VectorSpace(P, n)

A = matrix(P, [
    [0, 1],
    [1, 1]
])
b = V([0,0])

g = AffinePermutation(A, b)
cycles = g.get_cycles()
for c in cycles:
    print(c)

((0, 0),)
((1, 0), (0, 1), (1, 1))


Как и ожидалось данная подстанока имеет два цикла. Один цикл длины 3 и один цикл длины 1

Рассмотрим похожие примеры

In [24]:
P = GF(2)
n = 3
V = VectorSpace(P, n)

A = matrix(P, [
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 0]
])
b = V([0,0, 0])

g = AffinePermutation(A, b)
cycles = g.get_cycles()
for c in cycles:
    print(c)

((0, 0, 0),)
((1, 0, 0), (0, 0, 1), (0, 1, 0), (1, 0, 1), (0, 1, 1), (1, 1, 1), (1, 1, 0))


In [28]:
P = GF(3)
n = 3
V = VectorSpace(P, n)

A = matrix(P, [
    [0, 1, 0],
    [0, 0, 1],
    [2, 2, 1]
])
b = V([0,0, 0])

g = AffinePermutation(A, b)
cycles = g.get_cycles()
for c in cycles:
    print(c)

((0, 0, 0),)
((1, 0, 0), (0, 0, 2), (0, 2, 2), (2, 2, 0), (2, 0, 2), (0, 2, 0), (2, 0, 1), (0, 1, 2), (1, 2, 1), (2, 1, 1), (1, 1, 1), (1, 1, 2), (1, 2, 0), (2, 0, 0), (0, 0, 1), (0, 1, 1), (1, 1, 0), (1, 0, 1), (0, 1, 0), (1, 0, 2), (0, 2, 1), (2, 1, 2), (1, 2, 2), (2, 2, 2), (2, 2, 1), (2, 1, 0))


Заметим, что группа прибавления элементов векторного пространства (то есть правое регулярное представление группы $(V, +)$) также лежит в $AGL(n, q)$. Рассмотрим подстановки из данной группы

In [29]:
P = GF(2)
n = 3
V = VectorSpace(P, n)

A = matrix(P, [
    [1, 0, 0],
    [0, 1, 0],
    [1, 0, 1]
])
b = V([0,1, 0])

g = AffinePermutation(A, b)
cycles = g.get_cycles()
for c in cycles:
    print(c)

((0, 0, 0), (0, 1, 0))
((1, 0, 0), (1, 1, 1))
((1, 1, 0), (1, 0, 1))
((0, 0, 1), (0, 1, 1))


Используя построенную выше нумерацию для элементов векторных пространств, выведем циклы с номерами элементов

In [31]:
P = GF(2)
n = 3
V = VectorSpace(P, n)

A = matrix(P, [
    [1, 0, 0],
    [0, 1, 0],
    [1, 0, 1]
])
b = V([0,1, 0])

g = AffinePermutation(A, b)
cycles = g.get_cycles(numeric_form=True)
for c in cycles:
    print(c)

(0, 2)
(1, 7)
(3, 5)
(4, 6)


In [32]:
P = GF(2)
n = 3
V = VectorSpace(P, n)

A = matrix(P, [
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 0]
])
b = V([0,0, 0])

g = AffinePermutation(A, b)
cycles = g.get_cycles(numeric_form=True)
for c in cycles:
    print(c)

(0,)
(1, 4, 2, 5, 6, 7, 3)


Перебрав все подстановки из $AGL(2, 2)$ перечислим их  цикловые типы.

In [127]:

P = GF(2)
n = 2
V = VectorSpace(P, n)
Pnn = MatrixSpace(P, n, n)
for A in Pnn:
    if not A.is_invertible():
        continue
    for b in V:
        g = AffinePermutation(A, b)
        cycles = g.get_cycles()
        lens = [len(c) for c in cycles]
        print(lens)

[1, 1, 1, 1]
[2, 2]
[2, 2]
[2, 2]
[1, 2, 1]
[4]
[4]
[2, 1, 1]
[1, 3]
[3, 1]
[3, 1]
[3, 1]
[1, 1, 2]
[2, 1, 1]
[4]
[4]
[1, 2, 1]
[4]
[2, 1, 1]
[4]
[1, 3]
[3, 1]
[3, 1]
[3, 1]


Видим, что имеется много одинаковых цикловых типов. Перечислим без повторения. Для этого испозуем класс `Counter` из стандартной библиотеки

In [42]:
from collections import Counter

Данный класс (как следует из названия) предназначен для подсчета элементов

In [44]:
Counter([1, 2, 2, 3, 3, 4, 4, 4])

Counter({4: 3, 2: 2, 3: 2, 1: 1})

То есть 4 встречается 3 раза, 2 встречается 2 раза, 3 также 2 раза и 1 ровно 1 раз

Перепишем код выше

In [47]:
P = GF(2)
n = 2
V = VectorSpace(P, n)
Pnn = MatrixSpace(P, n, n)

cycle_types = set()

for A in Pnn:
    if not A.is_invertible():
        continue
    for b in V:
        g = AffinePermutation(A, b)
        cycles = g.get_cycles()
        lens = [len(c) for c in cycles]
        clens = Counter(lens)
        clens = sorted(clens.items())
        cycle_types.add(tuple(clens))
for  c in cycle_types:
    print(c)

((4, 1),)
((1, 4),)
((2, 2),)
((1, 2), (2, 1))
((1, 1), (3, 1))


Ответьте на вопрос все ли цикловые типы подстановок из всей симметрической группы $S_4$ встречаются среди перечисленных? Как вы можете это объяснить? 

In [ ]:
## YOUR CODE HERE

Оформите задачу вычисления всех цикловых типов в виде отдельной функции. Вычислите всевозможные цикловые типы аффинных подстановок для случая $p=2, n=3$ и $p=3, n=2$. Совпадает ли в данных случаях множество цикловых типов с множеством цикловых типов всех подстановок из $S_{8}$ и $S_{9}$, соответственно?

In [ ]:
## YOUR CODE HERE

Проделайте то же задание для случая $P = GF(4), n = 2$ и ответьте на предыдущие вопросы

In [ ]:
## YOUR CODE HERE

### Использование встроенных функций Sage

Ранее мы строили группу $AGL$, перебирая обратимые матрицы и векторы. Отметим, что уже при $p=2, n=8$ данный подход имеет неприемлемо высокую сложность. В связи с этим в данном разделе мы будем использовать другой подход, базирующийся на результатах из теории групп

Сначала воспользуемся предыдущим способом и рассмотрим границы его применимости. Для этого построим аффинную группу небольшого размера, породим подстановками из данной группы подгруппу в соответствующей симметрической группе и сравним мощности.

Для работы с симметрической группой будем пользоваться классом `SymmetricGroup`

In [51]:
S = SymmetricGroup(3)
for g in S:
    print(g)

()
(1,3,2)
(1,2,3)
(2,3)
(1,3)
(1,2)


Обратите внимание, что подстановки записываются циклами, при этом циклы длины $1$ не пишутся

Для наших целей будет удобнее работать с группой подстановок не на множестве $\{1,2,\ldots, q^n\}$, а на множестве $\{0,1,\ldots, q^n - 1\}$. Создать такое множество можно командой `list(range(n))`

In [52]:
list(range(4))

[0, 1, 2, 3]

In [53]:
S = SymmetricGroup(list(range(3)))
for g in S:
    print(g)

()
(0,2,1)
(0,1,2)
(1,2)
(0,2)
(0,1)


Для построения подстановки нужно передать ее цикловую запись

In [57]:
S([(0,1), (2,)])

(0,1)

In [62]:
P = GF(2)
n = 3
V = VectorSpace(P, n)

A = matrix(P, [
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 0]
])
b = V([0,0, 0])

g = AffinePermutation(A, b)
cycles = g.get_cycles(numeric_form=True)


S = SymmetricGroup(list(range(8)))
g = S(cycles)
print(g, type(g))

(1,4,2,5,6,7,3) <class 'sage.groups.perm_gps.permgroup_element.SymmetricGroupElement'>


In [61]:
A = matrix(P, [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])
b = V([0,1, 0])

g = AffinePermutation(A, b)
cycles = g.get_cycles(numeric_form=True)


S = SymmetricGroup(list(range(8)))
g = S(cycles)
print(g, type(g))

(0,2)(1,3)(4,6)(5,7) <class 'sage.groups.perm_gps.permgroup_element.SymmetricGroupElement'>


Построим $AGL(3, 2)$. Для этого сначала вычислим разложение на циклы всех подстановок

In [63]:
P = GF(2)
n = 3
V = VectorSpace(P, n)
Pnn = MatrixSpace(P, n, n)

perms = []
for A in Pnn:
    if not A.is_invertible():
        continue
    for b in V:
        g = AffinePermutation(A, b)
        cycles = g.get_cycles(numeric_form=True)
        perms.append(cycles)
        

Покажем первые 5 разложений

In [65]:
perms[:5]

[[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,)],
 [(0, 1), (2, 3), (4, 5), (6, 7)],
 [(0, 2), (1, 3), (4, 6), (5, 7)],
 [(0, 3), (1, 2), (4, 7), (5, 6)],
 [(0, 4), (1, 5), (2, 6), (3, 7)]]

Далее используя функцию `subgroup` сгенерируем подгруппу в симметрической группу

In [66]:
S = SymmetricGroup(list(range(8)))
agl = S.subgroup(perms)

Вычислим мощность данной группы

In [67]:
agl.cardinality()

1344

Для проверки того, что мы все слелали правильно сравним с теоретическим значением

In [71]:
def agl_cardinality(n, q):
    return np.prod([q^n - q^i for i in range(n)]) * q^n

In [88]:
agl_cardinality(3, 2)

1344

Проверим данный подход для случая $q=4, n=2$

In [73]:
q=4
n = 2

P = GF(q)
V = VectorSpace(P, n)
Pnn = MatrixSpace(P, n, n)

perms = []
for A in Pnn:
    if not A.is_invertible():
        continue
    for b in V:
        g = AffinePermutation(A, b)
        cycles = g.get_cycles(numeric_form=True)
        perms.append(cycles)
        
S = SymmetricGroup(list(range(q^n)))
agl = S.subgroup(perms)

In [74]:
print(agl.cardinality(), agl_cardinality(n, q))

2880 2880


Видим, что данные значения совпадают

Постройте указанным выше способом группу $AGL(2, 3)$  и вычислите ее мощность

In [75]:
## YOUR code here

~~Постройте~~ Попробуйте построить указанным выше способом группу $AGL(4, 2)$  и вычислите ее мощность. Прервать вычисления можно с помощью влкадки `kernel`

In [ ]:
## YOUR code here

Удалось ли выполнить построение?

In [ ]:
## YOUR code here

Однако как мы отмечали выше, данный подход, базирующийся на переборе матриц ограничен из-за высокой трудоемкости.

Поэтому мы воспользуемся тем фактом, что аффинную группу можно построить как нормализатор соответствующей группы в симметрической группе

Построим подстановки из группы сдвигов векторного пространства

In [76]:
q = 2
n = 4

P = GF(q)
V = VectorSpace(P, n)

A = matrix(P, [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
])



perms = []
for b in V:
    g = AffinePermutation(A, b)
    cycles = g.get_cycles(numeric_form=True)
    perms.append(cycles)




То есть мы сгенерировали все подстановки соответствующие прибавлению векторов векторного пространства 

In [80]:
for c in perms:
    print(c)

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,)]
[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11), (12, 13), (14, 15)]
[(0, 2), (1, 3), (4, 6), (5, 7), (8, 10), (9, 11), (12, 14), (13, 15)]
[(0, 3), (1, 2), (4, 7), (5, 6), (8, 11), (9, 10), (12, 15), (13, 14)]
[(0, 4), (1, 5), (2, 6), (3, 7), (8, 12), (9, 13), (10, 14), (11, 15)]
[(0, 5), (1, 4), (2, 7), (3, 6), (8, 13), (9, 12), (10, 15), (11, 14)]
[(0, 6), (1, 7), (2, 4), (3, 5), (8, 14), (9, 15), (10, 12), (11, 13)]
[(0, 7), (1, 6), (2, 5), (3, 4), (8, 15), (9, 14), (10, 13), (11, 12)]
[(0, 8), (1, 9), (2, 10), (3, 11), (4, 12), (5, 13), (6, 14), (7, 15)]
[(0, 9), (1, 8), (2, 11), (3, 10), (4, 13), (5, 12), (6, 15), (7, 14)]
[(0, 10), (1, 11), (2, 8), (3, 9), (4, 14), (5, 15), (6, 12), (7, 13)]
[(0, 11), (1, 10), (2, 9), (3, 8), (4, 15), (5, 14), (6, 13), (7, 12)]
[(0, 12), (1, 13), (2, 14), (3, 15), (4, 8), (5, 9), (6, 10), (7, 11)]
[(0, 13), (1, 12), (2, 15), (3, 14), (4, 9), 

Далее этими подстановками породим соотвествующую группу

In [81]:
S = SymmetricGroup(list(range(q^n)))
T = S.subgroup(perms)

Посмотрим на свойства этой группы

In [82]:
T.is_abelian()

True

In [83]:
T.is_regular()

True

In [84]:
T.cardinality()

16

In [85]:
T.exponent()

2

А теперь самое интересное. Построим аффинную группу $AGL(4, 2)$.

In [86]:
agl = S.normalizer(T)

Проверим, не ошиблись ли мы

In [89]:
agl.cardinality()

322560

In [90]:
agl_cardinality(4, 2)

322560

Видим, что построение выполняется очень быстро. Построим таким же образом $AGL(8, 2)$. Сначала построим группу сдвигов. Следующая ячейка может выполняться в течение пары минут

In [91]:
q = 2
n = 8

P = GF(q)
V = VectorSpace(P, n)

A = identity_matrix(P, n)

perms = []
for b in V:
    g = AffinePermutation(A, b)
    cycles = g.get_cycles(numeric_form=True)
    perms.append(cycles)

In [92]:
S = SymmetricGroup(list(range(q^n)))
T = S.subgroup(perms)

Далее построим $AGL(8,2)$

In [93]:
agl = S.normalizer(T)

In [94]:
agl.cardinality()

1369104324918194995200

Более того можно найти экспоненту группу.

In [102]:
agl.exponent()

337322160

Напишите функцию для нахождения  экспоненты аффинной группы и найдите экспоненту группы $AGL(n, 2)$ для $n=5,6,7$ и экспоненту групп $AGL(n, 3)$ для $n=3,4,5$

In [ ]:
### your code here

Проделайте следующее задание

Постройте аффинную группу $AGL(2, 4)$ перебором матриц и векторов и вычислите ее мощность

In [103]:
q=4
n = 2

P = GF(q)
V = VectorSpace(P, n)
Pnn = MatrixSpace(P, n, n)

perms = []
for A in Pnn:
    if not A.is_invertible():
        continue
    for b in V:
        g = AffinePermutation(A, b)
        cycles = g.get_cycles(numeric_form=True)
        perms.append(cycles)
        
S = SymmetricGroup(list(range(q^n)))
agl = S.subgroup(perms)

In [104]:
agl.cardinality()

2880

Далее постройте группу равную нормализатору группы сдвигов векторного пространства $V = {}_PP^2$, где $P = GF(4)$. Вычислите мощность данного нормализатора

In [105]:
q = 4
n = 2

P = GF(q)
V = VectorSpace(P, n)

A = identity_matrix(P, n)

perms = []
for b in V:
    g = AffinePermutation(A, b)
    cycles = g.get_cycles(numeric_form=True)
    perms.append(cycles)
    
S = SymmetricGroup(list(range(q^n)))
T = S.subgroup(perms)

agl = S.normalizer(T)


In [106]:
agl.cardinality()

322560

Как вы можете объяснить данный эффект?

In [ ]:
## YOUR CODE HERE

Возникает вопрос. Как построить группу $GL(n, q)$? Для небольших значений можно приметь указанный выше подход с перебором матриц. Например построим $GL(3, 2)$.

In [109]:
V([0, 0, 0])

(0, 0, 0)

In [110]:
q=2
n = 3

P = GF(q)
V = VectorSpace(P, n)
b = V([0,0,0])
Pnn = MatrixSpace(P, n, n)

perms = []
for A in Pnn:
    if not A.is_invertible():
        continue
    g = AffinePermutation(A, b)
    cycles = g.get_cycles(numeric_form=True)
    perms.append(cycles)
        
S = SymmetricGroup(list(range(q^n)))
gl = S.subgroup(perms)

In [112]:
gl.cardinality()

168

In [113]:
gl.exponent()

84

Для проверки напишите функцию для вычисления мощности группы $GL(n, q)$

In [114]:
def gl_cardinality(n, q):
    return np.prod([q^n - q^i for i in range(n)])

In [115]:
gl_cardinality(3, 2)

168

Для значений $n\geq 5$ данный подход также неприемлем. Однако построить группу $GL(n, q)$ можно как стабилизатор нулевого элемента. Проделаем это для построения $GL(4, 2)$.

In [116]:
q = 2
n = 4

P = GF(q)
V = VectorSpace(P, n)

A = identity_matrix(P, n)

perms = []
for b in V:
    g = AffinePermutation(A, b)
    cycles = g.get_cycles(numeric_form=True)
    perms.append(cycles)
    
S = SymmetricGroup(list(range(q^n)))
T = S.subgroup(perms)

agl = S.normalizer(T)

In [120]:
gl = agl.stabilizer(0)

In [121]:
gl.cardinality()

20160

In [122]:
gl_cardinality(4, 2)

20160

In [123]:
gl.exponent()

420

Вычислите экспоненту группы $GL(n, 2)$ для $n=4,5,7,8$. Сравните с результатами для $AGL(n,2)$. 

In [125]:
## your code here

Напишите выводы о проделанной работе.

In [126]:
## your code here

### Задание для молодцов =)

Предложите способ построения группы $AGL(n, q)$ для случая непростых значений $q$ отличный от перебора матриц и веторов. Реализуйте данный способ. 

**Подсказка**. Чем порождается группа $GL(n, q)$? Что такое элементарная матрица?

In [ ]:
## your code here